In [1]:
import os

import numpy as np
import pandas as pd
from kaggle_datasets import KaggleDatasets
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras.applications.efficientnet as efn

## Helper functions

The following functions are hidden:
```python
auto_select_accelerator()

build_decoder(with_labels=True, target_size=(256, 256), ext='jpg')

build_augmenter(with_labels=True)

build_dataset(paths, labels=None, bsize=32, cache=True,
              decode_fn=None, augment_fn=None,
              augment=True, repeat=True, shuffle=1024, 
              cache_dir="")
```

Unhide below to see:

In [2]:
def auto_select_accelerator():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    
    return strategy


def build_decoder(with_labels=True, target_size=(300, 300), ext='jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path)
        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels=3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels=3)
        else:
            raise ValueError("Image extension not supported")

        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        return img
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    return decode_with_labels if with_labels else decode


def build_augmenter(with_labels=True):
    def augment(img):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        return img
    
    def augment_with_labels(img, label):
        return augment(img), label
    
    return augment_with_labels if with_labels else augment


def build_dataset(paths, labels=None, bsize=32, cache=True,
                  decode_fn=None, augment_fn=None,
                  augment=True, repeat=True, shuffle=1024, 
                  cache_dir=""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)
    
    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)
    
    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize).prefetch(AUTO)
    
    return dset

## Variables and configurations

In [3]:
COMPETITION_NAME = "ranzcr-clip-catheter-line-classification"
strategy = auto_select_accelerator()
BATCH_SIZE = strategy.num_replicas_in_sync * 16
# GCS_DS_PATH = KaggleDatasets().get_gcs_path(COMPETITION_NAME)

Running on 1 replicas


## Preparing dataset

### Loading and preprocess CSVs

In [4]:
########################### GPU ##########################
load_dir = f"/kaggle/input/{COMPETITION_NAME}/"
df = pd.read_csv(load_dir + 'train.csv')
paths = load_dir + "train/" + df['StudyInstanceUID'] + '.jpg'

sub_df = pd.read_csv(load_dir + 'sample_submission.csv')
test_paths = load_dir + "test/" + sub_df['StudyInstanceUID'] + '.jpg'

# Get the multi-labels
label_cols = sub_df.columns[1:]
labels = df[label_cols].values

In [5]:
################################### TPU ##########################
# load_dir = f"/kaggle/input/{COMPETITION_NAME}/"
# df = pd.read_csv(load_dir + 'train.csv')

# # paths = load_dir + "train/" + df['StudyInstanceUID'] + '.jpg'
# paths = GCS_DS_PATH + "/train/" + df['StudyInstanceUID'] + '.jpg'

# sub_df = pd.read_csv(load_dir + 'sample_submission.csv')

# # test_paths = load_dir + "test/" + sub_df['StudyInstanceUID'] + '.jpg'
# test_paths = GCS_DS_PATH + "/test/" + sub_df['StudyInstanceUID'] + '.jpg'

# # Get the multi-labels
# label_cols = sub_df.columns[1:]
# labels = df[label_cols].values

In [6]:
# paths=paths[0:12000]

# labels=labels[0:12000]
len(paths)

30083

In [7]:
# Train test split
(
    train_paths, valid_paths, 
    train_labels, valid_labels
) = train_test_split(paths, labels, test_size=0.2, random_state=42)

In [8]:
# Build the tensorflow datasets
IMSIZES = (224, 240, 260, 300, 320,340,480, 528, 600)
# index i corresponds to b-i
size = IMSIZES[4]

decoder = build_decoder(with_labels=True, target_size=(size, size))
test_decoder = build_decoder(with_labels=False, target_size=(size, size))

# Build the tensorflow datasets
dtrain = build_dataset(
    train_paths, train_labels, bsize=BATCH_SIZE, 
    cache_dir='/kaggle/tf_cache', decode_fn=decoder
)

dvalid = build_dataset(
    valid_paths, valid_labels, bsize=BATCH_SIZE, 
    repeat=False, shuffle=False, augment=False, 
    cache_dir='/kaggle/tf_cache', decode_fn=decoder
)

dtest = build_dataset(
    test_paths, bsize=BATCH_SIZE, repeat=False, 
    shuffle=False, augment=False, cache=False, 
    decode_fn=test_decoder
)

## Modeling

In [9]:
# model_path_b4 = '../input/tfkeras-efficientnet-weights/efficientnetb4_notop.h5'  # imagenet
# n_labels = labels.shape[1]

# with strategy.scope():
#     model_B4= tf.keras.Sequential([
#         efn.EfficientNetB4(
#             input_shape=(size, size, 3),
#             weights=model_path_b4,
#             include_top=False,
#             drop_connect_rate=0.5),
#         tf.keras.layers.GlobalAveragePooling2D(),
#         tf.keras.layers.Dense(n_labels, activation='sigmoid')
#     ])
#     model.compile(
#         optimizer='adam',
#         loss='binary_crossentropy',
#         metrics=[tf.keras.metrics.AUC(multi_label=True)])
#     model.summary()

In [10]:
n_labels = labels.shape[1]

with strategy.scope():
    model_Xception= tf.keras.Sequential([
        tf.keras.applications.Xception(
            input_shape=(size, size, 3),
            weights='imagenet',
            include_top=False,
        ),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(n_labels, activation='sigmoid')
    ])
    model_Xception.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=[tf.keras.metrics.AUC(multi_label=True)])
    model_Xception.summary()

83683744/83683744 [==============================] - 3s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 10, 10, 2048)      20861480  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 11)                22539     
                                                                 
Total params: 20,884,019
Trainable params: 20,829,491
Non-trainable params: 54,528
_________________________________________________________________


In [11]:
steps_per_epoch = train_paths.shape[0] // BATCH_SIZE
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'model2.h5', save_best_only=True, monitor='val_auc', mode='max')
lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_auc", patience=3, min_lr=1e-6, mode='max')

In [ ]:
history = model_Xception.fit(
    dtrain, 
    epochs=30,
    verbose=1,
    callbacks=[checkpoint, lr_reducer],
    steps_per_epoch=steps_per_epoch,
    validation_data=dvalid)

Epoch 1/30
1504/1504 [==============================] - 1946s 1s/step - loss: 0.2342 - auc: 0.7720 - val_loss: 0.2359 - val_auc: 0.7953 - lr: 0.0010
Epoch 2/30
1504/1504 [==============================] - 487s 324ms/step - loss: 0.2137 - auc: 0.8152 - val_loss: 0.2350 - val_auc: 0.8107 - lr: 0.0010
Epoch 3/30
1504/1504 [==============================] - 456s 303ms/step - loss: 0.1998 - auc: 0.8477 - val_loss: 0.1959 - val_auc: 0.8592 - lr: 0.0010
Epoch 4/30
1504/1504 [==============================] - 457s 304ms/step - loss: 0.1903 - auc: 0.8709 - val_loss: 0.1941 - val_auc: 0.8768 - lr: 0.0010
Epoch 5/30
1504/1504 [==============================] - 455s 303ms/step - loss: 0.1827 - auc: 0.8833 - val_loss: 0.2147 - val_auc: 0.8894 - lr: 0.0010
Epoch 6/30
1504/1504 [==============================] - 456s 303ms/step - loss: 0.1765 - auc: 0.8938 - val_loss: 0.1838 - val_auc: 0.9021 - lr: 0.0010
Epoch 7/30
1504/1504 [==============================] - 456s 303ms/step - loss: 0.1704 - auc: 0.

In [14]:
model_Xception.save('Xception_full_data.h5')